In [1]:
# Loading libraries
library(readr)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Before we start, let's define some functions that will be useful later in this project. Our original dataset has the data for maximum temperature and cumulative rain for 1 hour intervals. But that is not what we want, we want this data for 24 hour intervals. So, the first function will calculate a cumulative sum for a specified column on the Data Frame for a specified window size, whuch we will use for the rain, and the second function picks the maximum value for a specified column and window size, which we will use for the maximum temperature.

In [2]:

# Functions
cumulative_sum <- function(data, column_name, window_size) {
  n_rows <- nrow(data)
  result <- numeric(n_rows)
  #progress bar
  pb <- txtProgressBar(min = 0,
                       max = n_rows,
                       style = 3,
                       width = 50,
                       char = "=")
  for (i in window_size:n_rows) {
    result[i] <- sum(data[[column_name]][i:(i - window_size + 1)])
    setTxtProgressBar(pb, i)
  }
  cat("\nDone!")
  return(result)
}

max_within_window <- function(data, column_name, window_size) {
  n_rows <- nrow(data)
  result <- numeric(n_rows)
  pb <- txtProgressBar(min = 0,
                       max = n_rows,
                       style = 3,
                       width = 50,
                       char = "=")  
  for (i in (window_size + 1):n_rows) {
    result[i] <- max(data[[column_name]][i:(i - window_size + 1)])
    setTxtProgressBar(pb, i)    
  }
  cat("\nDone!")
  return(result)
}

Now we can load our data.

In [3]:
dados <- read_csv("/kaggle/input/RJweather/data.csv",show_col_types = FALSE )

We don't need the entire dataset for this project. So, we create a subset containing only the three weather stations selected (A602, A621 and A652) and the desired timeframe (from 2012/01/01 to 2021/12/31). We will also discard all the data for the days where there is a missing value in any of the columns we are working with.

In [4]:
selected_columns <- c("primary_key", "id_estacao", "data", "horario", "temperatura_maxima", "acumulado_chuva_1_h")

# Subset the dataframe based on conditions
subset_dados <- dados[dados$id_estacao %in% c("A602", "A621", "A652") &
                        dados$data >= as.Date("2012-01-01") & dados$data <= as.Date("2021-12-31"),
                      selected_columns]

# Identify dates with missing data for specified columns
missing_values <- unique(subset_dados$data[is.na(subset_dados$temperatura_maxima) | is.na(subset_dados$acumulado_chuva_1_h)])

# Filter out rows with missing data for specific columns and the corresponding dates
DataSet <- subset_dados[!(subset_dados$data %in% missing_values), ]

Finally, we used the functions we defined before to calculate the daily data, instead of hourly, and keep only one register per date.

In [5]:
# Calculate rains above 30mm in 24h (Cumulative sum function)
window_size_rain <- 24
column_name_rain <- "acumulado_chuva_1_h"
DataSet$rain_24h <- cumulative_sum(DataSet, column_name_rain, window_size_rain)

# Calculate extreme heat events (Calculate max within window function)
window_size_temp <- 24
column_name_temp <- "temperatura_maxima"
DataSet$max_temp_24h <- max_within_window(DataSet, column_name_temp, window_size_temp)

DataSet <- DataSet %>%
  distinct(data, id_estacao, .keep_all = TRUE)
DataSet$year<-as.numeric(substr(DataSet$data,1,4))

  |==================================================| 100%
  |==================================================| 100%
Done!

Next, we create new datasets where we will count the events we are monitoring. In dc30 we will count the number of days with cumulative rain above 30mm, and in mt40 we count the days with maximum temparature above 40ºC.

In [6]:
dc30 <- data.frame(estacao = character(0), ano = numeric(0), rain_over_30 = numeric(0))
stations <- c("A602", "A621", "A652")
years <- seq(2012, 2021)

for (year in years) {
  for (station in stations) {
    dc_row <- data.frame(
      estacao = station,
      ano = year,
      rain_over_30 = sum(DataSet$id_estacao == station & DataSet$rain_24h > 30 & DataSet$year == year)
    )
    dc30 <- rbind(dc30, dc_row)
  }
}

mt40 <- data.frame(estacao = character(0), ano = numeric(0), temp_over_40 = numeric(0))
stations <- c("A602", "A621", "A652")
years <- seq(2012, 2021)

for (year in years) {
  for (station in stations) {
    mt_row <- data.frame(
      estacao = station,
      ano = year,
      temp_over_40 = sum(DataSet$id_estacao == station & DataSet$max_temp_24h > 40 & DataSet$year == year)
    )
    mt40 <- rbind(mt40, mt_row)
  }
}

To realize the Chi-square goodness of fit test, it is necessary to calculate the probability of each observed outcome, assuming that the data follows a Poisson distribution. But in order to do that, first we need to calculate λ. In this case, we can consider the sample mean as an estimator. After that, we can calculate the probability of the observed outcomes using the R function dpois. We store the results in matrices.

In [7]:
#Calculating probabilities for dc30
lambda_dc30 = c()

for (station in stations) {
  lambda_dc30<-append(lambda_dc30,mean(dc30$rain_over_30[dc30$estacao==station]))
}

probs_dc30 <- matrix(nrow = length(stations), ncol = length(years))
rownames(probs_dc30) <- stations
colnames(probs_dc30) <- years

i <- 1
for (station in stations) {
  probs_dc30[i, ] <- dpois(dc30$rain_over_30[dc30$estacao == station], lambda_dc30[i])
  i <- i + 1
}

#Calculating probabilities for mt40
lambda_mt40 = c()

for (station in stations) {
  lambda_mt40<-append(lambda_mt40,mean(mt40$temp_over_40[mt40$estacao==station]))
}

probs_mt40 <- matrix(nrow = length(stations), ncol = length(years))
rownames(probs_mt40) <- stations
colnames(probs_mt40) <- years

i <- 1
for (station in stations) {
  probs_mt40[i, ] <- dpois(mt40$temp_over_40[mt40$estacao == station], lambda_mt40[i])
  i <- i + 1
}

In [8]:
chis_dc30 <- c()
p_vals_dc30 <- c()
i <- 1

for (station in stations) {
  tryCatch({
    test <- chisq.test(dc30$rain_over_30[dc30$estacao == station], p = probs_dc30[i,], rescale.p = TRUE)
    chis_dc30 <- append(chis_dc30, test$statistic)
    p_vals_dc30 <- append(p_vals_dc30, test$p.value)
    i <- i + 1
  }, error = function(e) {
    chis_dc30 <- append(chis_dc30, NA)  # Store NA for the failed calculation
    p_vals_dc30 <- append(p_vals_dc30, NA)
    i <- i + 1
    message(paste("Error for station", station, ":", conditionMessage(e)))
  })
}

# Similar error handling for chis_mt40 and p_vals_mt40 loops


chis_mt40 = c()
p_vals_mt40 = c()
i <- 1
for (station in stations) {
  tryCatch({
    test <- chisq.test(mt40$temp_over_40[mt40$estacao == station], p = probs_mt40[i,], rescale.p = TRUE)
    chis_mt40 <- append(chis_mt40, test$statistic)
    p_vals_mt40 <- append(p_vals_mt40, test$p.value)
    i <- i + 1
  }, error = function(e) {
    chis_mt40 <- append(chis_mt40, NA)  # Store NA for the failed calculation
    p_vals_mt40 <- append(p_vals_mt40, NA)
    i <- i + 1
    message(paste("Error for station", station, ":", conditionMessage(e)))
  })
}

Warning message in chisq.test(dc30$rain_over_30[dc30$estacao == station], p = probs_dc30[i, :
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(dc30$rain_over_30[dc30$estacao == station], p = probs_dc30[i, :
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(dc30$rain_over_30[dc30$estacao == station], p = probs_dc30[i, :
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(mt40$temp_over_40[mt40$estacao == station], p = probs_mt40[i, :
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(mt40$temp_over_40[mt40$estacao == station], p = probs_mt40[i, :
“Chi-squared approximation may be incorrect”
Error for station A652 : at least one entry of 'x' must be positive



It seems that we have a problem with the Chi-Squared Test for the mt40 matrix. Let's see what happened.

In [9]:
print(mt40[mt40$estacao==station,])

   estacao  ano temp_over_40
3     A652 2012            0
6     A652 2013            0
9     A652 2014            0
12    A652 2015            0
15    A652 2016            0
18    A652 2017            0
21    A652 2018            0
24    A652 2019            0
27    A652 2020            0
30    A652 2021            0


![](https://i.kym-cdn.com/photos/images/original/000/228/269/demotivational-posters-theres-your-problem.jpg)

We got an error because the station A652 didn't record any temperature over 40ºC in the selected timeframe. No problem, we can ignore this station then.
In the end, we got these results:

**Results for Rain over 30mm in 24h:**

| Station   | χ²          | P-value    |
|-----------|-------------|------------|
| A602 | 5.713608e+15 | 0 |
| A621 | 1.518308e+11 | 0 |
| A652 | 5.739998e+11 | 0 |


**Results for Maximum Temperature over 40ºC:**

| Station   | χ²          | P-value    |
|-----------|-------------|------------|
| A602 | 136699652 | 0 |
| A621 | 1983807 | 0 |
| A652 | NA* | NA* |

**Chi-Squared GOF test failed as no such events were registered in the given timeframe.*

So, what can we conclude from this test? In all the evaluated scenarios, it was possible to reject the null hypothesis, meaning that none of the observed samples followed a Poisson distribution. The Poisson distribution is a relatively simple model that assumes that the events being modeled are independent and occur at a constant rate. In practice, the occurrence of extreme weather events may be influenced by a variety of factors, such as atmospheric and oceanic conditions, which may not be well-represented by the Poisson distribution. As a result, more complex statistical models are needed to accurately model extreme weather events.